In [80]:
import requests
from pprint import PrettyPrinter
import tensorflow as tf
import json
import base64

In [81]:
pp = PrettyPrinter()
pp.pprint(requests.get('https://emotion-nlp-model-production.up.railway.app/v1/models/emotion-model').json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1706726960'}]}


In [82]:
def prepare_json(text, example_id=1):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)).SerializeToString()

    result = [{'examples': {'b64': base64.b64encode(example).decode()}}]

    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": result
        }
    )


def get_prediction(text):
    response = requests.post(
        'https://emotion-nlp-model-production.up.railway.app/v1/models/emotion-model:predict',
        data=prepare_json(text)
    )
    
    predict = response.json()['predictions'][0]

    if predict[0] > 0.5:
        return 'anger'
    elif predict[1] > 0.5:
        return 'fear'
    elif predict[2] > 0.5:
        return 'joy'
    elif predict[3] > 0.5:
        return 'love'
    elif predict[4] > 0.5:
        return 'sadness'
    elif predict[5] > 0.5:
        return 'surprise'
    else:
        return 'unknown'

In [83]:
test_text = "I love you"

get_prediction(test_text)

'love'